# Test HAND processing and stitching

# Import modules and load DEM from datacube

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pysheds.grid import Grid

import pyproj
from affine import Affine
import xarray as xr
import os
import geopandas as gpd
from shapely.geometry import Point, Polygon
import rasterio

import warnings
warnings.filterwarnings('ignore')

In [2]:
from skimage.morphology import watershed
from scipy import ndimage

In [3]:
import datacube
prod_dc = datacube.Datacube()

In [4]:
def add_streams(grid, save=False):
    streamnetwork = gpd.read_file('geofabric/SH_Network_GDB_National_V3_0_5_Beta_MajorFiltered.shp')
    streamnetwork = streamnetwork.to_crs({'init': 'epsg:3577'})
    bbox = grid.bbox

    p1 = Point(bbox[0], bbox[3])
    p2 = Point(bbox[2], bbox[3])
    p3 = Point(bbox[2], bbox[1])
    p4 = Point(bbox[0], bbox[1])

    np1 = (p1.coords.xy[0][0], p1.coords.xy[1][0])
    np2 = (p2.coords.xy[0][0], p2.coords.xy[1][0])
    np3 = (p3.coords.xy[0][0], p3.coords.xy[1][0])
    np4 = (p4.coords.xy[0][0], p4.coords.xy[1][0])

    bb_polygon = Polygon([np1, np2, np3, np4])
    subset=streamnetwork.intersects(bb_polygon)
    
    result = rasterio.features.rasterize(streamnetwork[subset].geometry, all_touched=True,
                                     out_shape = grid.shape,
                                     transform = grid.affine)
    if save:
        with rasterio.open(
            "streamnetwork.tif",
            'w',
            driver='GTiff',
            width=grid.shape[1],
            height=grid.shape[0],
            count=1,
            dtype=np.uint8,
            nodata=0,
            transform=grid.affine,
            crs={'init': "EPSG:3577"}) as out:
                 out.write_band(1, result.astype(np.uint8))
            
    grid.add_gridded_data(result,'streams')
    #return result

In [5]:
dirmap=(4, 2, 1, 128, 64, 32, 16, 8)
routing='d8'

In [6]:
def fix_dem(grid):
    depressions = grid.detect_depressions('dem')
    n_depression = (depressions).sum()
    flats = grid.detect_flats('dem')
    n_flat = (flats).sum()
    i=0
    while n_depression > 0 or n_flat>0:
        print("fill depressions and resolve flats iter:", i, n_depression, n_flat)
        pre_depression = n_depression
        pre_flat = n_flat
    
        grid.fill_depressions(data='dem', out_name='dem')
        grid.resolve_flats('dem', out_name='dem')
    
        depressions = grid.detect_depressions('dem')
        n_depression = (depressions).sum()
        flats = grid.detect_flats('dem')
        n_flat = (flats).sum()
    
        if n_depression== pre_depression and n_flat==flat: break
        i+=1

In [7]:
def hand(dem, acc_thresh=2000, acc=False, dirmap=dirmap, routing=routing):
    coords = list(dem.coords.keys())
    if 'x' in coords: coord_x, coord_y = 'x', 'y' 
    else: coord_x, coord_y = 'longitude', 'latitude'
    
    dem['dem_h'] = dem.dem_h.where(dem.dem_h!=dem.dem_h.attrs['nodata'])
    dem['dem_h'] = dem.dem_h.where(dem.dem_h > -1000)
    xres = dem[coord_x].values[1]-dem[coord_x].values[0]
    yres = dem[coord_y].values[1]-dem[coord_y].values[0]
    x_ref = dem[coord_x].values[0]
    y_ref = dem[coord_y].values[0]
    
    grid = Grid(nodata = np.nan, 
            crs = pyproj.Proj('+init=epsg:%s'%dem.dem_h.attrs['crs'].epsg))
    grid.add_gridded_data(dem.dem_h.values.squeeze(),
                      data_name = 'dem', affine = Affine(xres, 0, x_ref, 0, yres, y_ref),
                      shape = dem.dem_h.values.squeeze().shape, 
                      nodata = np.nan, 
                      crs = pyproj.Proj('+init=epsg:%s'%dem.dem_h.attrs['crs'].epsg))
    fix_dem(grid)
    
    if acc:
        grid.flowdir(data='dem', out_name='dir',astype=np.float32, dirmap=dirmap, routing=routing)
#        add_streams(grid)
        grid.accumulation(data='dir', out_name='acc', pad=False, dirmap=dirmap, routing=routing, 
                          #weights = (grid.streams*100.+1)/101.,
                          astype=np.float32)
        if not acc_thresh: 
            acc_thresh = np.nanpercentile(grid.acc, 90)
            #print("accumulation threshold:", acc_thresh)
            #acc = grid.acc*grid.streams
            #acc_thresh = np.nanpercentile(acc[acc>0],1)
        print("accumulation threshold:", acc_thresh)
        print("99 percentile accumulation:", np.nanpercentile(grid.acc, 99))
        dem['acc'] = (coord_y, coord_x), grid.acc.astype(np.float32)
        dem['hand'] = (coord_y, coord_x), grid.compute_hand('dir', 'dem', grid.acc > acc_thresh, 
                                                            dirmap=dirmap, routing=routing, 
                                                            inplace=False,astype=np.float32)
        dem['dir'] = (coord_y, coord_x), grid.dir.astype(np.float32)
    else:
        add_streams(grid)
    
        dem['streams'] = (coord_y, coord_x), grid.streams#.astype(np.float32)
        markers = (dem.dem_h.where(dem.streams==1,0)).round()
        labels = watershed(dem.dem_h.values, markers.values, mask = ~np.isnan(dem.dem_h).values)
        dem_min = ndimage.minimum(dem.dem_h.values, labels=labels, index=labels)
        dem['hand'] = (coord_y, coord_x), dem.dem_h.values-dem_min
        #dem['hand'] = (coord_y, coord_x), grid.compute_hand('dir', 'dem', grid.streams > 0, 
        #                                                    dirmap=dirmap, routing=routing, 
        #                                                    inplace=False,astype=np.float32)
    
    


In [8]:
# tasmania

#tile_list = ['11,-46','11,-47','11,-48','12,-46','12,-47']

#tile_list = ['11,-47','12,-47']

In [9]:
tsize = 1e5
tsize_x = 2
tsize_y = 2

In [10]:
bbox = (-953243.9623141023, -2146122.18804447, -441593.96231410233, -1700652.18804447)
ymin, ymax = np.floor(bbox[1]/tsize), np.ceil(bbox[3]/tsize)
xmin, xmax = np.floor(bbox[0]/tsize), np.ceil(bbox[2]/tsize)

tile_x =  np.arange(xmin, xmax, tsize_x)
tile_y = np.arange(ymin, ymax, tsize_y)
tile_list = ['%d,%d'%(x,y) for x in tile_x for y in tile_y]
print(np.ceil((ymax-ymin)/tsize_y)*np.ceil((xmax-xmin)/tsize_x), len(tile_list))
print(tile_list)

9.0 9
['-10,-22', '-10,-20', '-10,-18', '-8,-22', '-8,-20', '-8,-18', '-6,-22', '-6,-20', '-6,-18']


In [11]:
tilesize = tsize_x*tsize
buffersize = 3000

#output_crs = 'EPSG:4326'
output_crs = 'EPSG:3577'

res = {'EPSG:3577':30, 'EPSG:4326':0.0002777777777822571046}

#dem = []
for tile in tile_list:
    outputname = 'hand_watershed/dem_hand_watershed_%s_%s.nc'%(tile.split(',')[0],tile.split(',')[1])
    if os.path.exists(outputname):continue
    x0, y0 = int(tile.split(',')[0])*100000, int(tile.split(',')[1])*100000
    x = (x0-buffersize, x0+tilesize+buffersize)
    y = (y0-buffersize, y0+tilesize+buffersize)
    x_output = (x0, x0+tilesize)
    y_output = (y0, y0+tilesize)
    dem_query = { 'y': y , 'x': x, 'crs': 'EPSG:3577', 
                 'output_crs':output_crs, 'resolution': (res[output_crs], res[output_crs]),
                }
    print(dem_query)
    dem_tile = prod_dc.load(product = 'srtm_dem1sv1_0', measurements=['dem_h'], **dem_query).isel(time=0)
    hand(dem_tile)

    del dem_tile['time']
    dem_tile.attrs['crs']=dem_tile.attrs['crs'].epsg
    del dem_tile['dem_h'].attrs['crs'], dem_tile['dem_h'].attrs['nodata']
    dem_tile.sel(x=slice(*x_output), y = slice(*y_output)).to_netcdf(outputname)
    #dem.append(dem_tile.sel(x=slice(*x_output), y = slice(*y_output)).copy())
    #if dem: dem = xr.concat([dem, dem_tile], dim='x')
    #else: dem = dem_tile.copy()
    del dem_tile
    #break

{'y': (-2203000, -1997000.0), 'x': (-1003000, -797000.0), 'crs': 'EPSG:3577', 'output_crs': 'EPSG:3577', 'resolution': (30, 30)}
fill depressions and resolve flats iter: 0 7178 44914
fill depressions and resolve flats iter: 1 1600 2825
fill depressions and resolve flats iter: 2 187 187
fill depressions and resolve flats iter: 3 54 124
{'y': (-2003000, -1797000.0), 'x': (-1003000, -797000.0), 'crs': 'EPSG:3577', 'output_crs': 'EPSG:3577', 'resolution': (30, 30)}
fill depressions and resolve flats iter: 0 14241 57025
fill depressions and resolve flats iter: 1 504 661
fill depressions and resolve flats iter: 2 269 269
fill depressions and resolve flats iter: 3 119 232
{'y': (-1803000, -1597000.0), 'x': (-1003000, -797000.0), 'crs': 'EPSG:3577', 'output_crs': 'EPSG:3577', 'resolution': (30, 30)}
fill depressions and resolve flats iter: 0 22630 28318
fill depressions and resolve flats iter: 1 1382 2819
fill depressions and resolve flats iter: 2 64 64
fill depressions and resolve flats iter:

In [12]:
len(tile_x), len(tile_y)

(3, 3)

In [13]:
import glob
import numpy as np
xs =[int(f.split('/')[-1].split('_')[3]) for f in glob.glob('hand_watershed/dem_hand_watershed_*.nc')]
xs = list(np.sort(np.unique(xs)))
ys =[int(f.split('/')[-1].split('.')[0].split('_')[4]) for f in glob.glob('hand_watershed/dem_hand_watershed_*.nc')]
ys = list(np.sort(np.unique(ys)))
dem_m = xr.concat([xr.open_mfdataset(glob.glob('hand_watershed/dem_hand_watershed_%d_*.nc'%x)) for x in xs], dim='x')

In [14]:
dem_m

<xarray.Dataset>
Dimensions:  (x: 20000, y: 20000)
Coordinates:
  * y        (y) float64 -2.2e+06 -2.2e+06 -2.2e+06 ... -1.6e+06 -1.6e+06
  * x        (x) float64 -1e+06 -9.999e+05 -9.999e+05 ... -4e+05 -4e+05
Data variables:
    dem_h    (y, x) float32 dask.array<shape=(20000, 20000), chunksize=(6666, 6666)>
    streams  (y, x) uint8 dask.array<shape=(20000, 20000), chunksize=(6666, 6666)>
    hand     (y, x) float32 dask.array<shape=(20000, 20000), chunksize=(6666, 6666)>
Attributes:
    crs:      3577

In [15]:
dem_m.to_netcdf('hand_watershed/mosaic.nc',compute=True)
dem_m.close()

In [16]:
from datacube import helpers
from datacube.utils import geometry

dem = xr.open_dataset('hand_watershed/mosaic.nc')
dem.attrs['crs'] = geometry.CRS('epsg:%s'%dem.attrs['crs'])
for var in list(dem.data_vars):
    ds = dem[var].to_dataset(name=var)
    ds.attrs['crs'] = dem.attrs['crs']
    helpers.write_geotiff('hand_watershed/mosaic_%s.tif'%var, ds)


In [17]:
!ls hand_watershed 

dem_hand_watershed_-10_-18.nc  dem_hand_watershed_-6_-22.nc  mosaic_hand.tif
dem_hand_watershed_-10_-20.nc  dem_hand_watershed_-8_-18.nc  mosaic.nc
dem_hand_watershed_-10_-22.nc  dem_hand_watershed_-8_-20.nc  mosaic_streams.tif
dem_hand_watershed_-6_-18.nc   dem_hand_watershed_-8_-22.nc
dem_hand_watershed_-6_-20.nc   mosaic_dem_h.tif
